# Magnetic

- Some magnetic and some neutral dominos are put together to a rectangle.
- Each magnetic domino has two poles, one positive pole (+) and one negative pole (–). Each neutral domino has two neutral poles (x).
- Two positive or two negative poles cannot touch orthogonally.
- A number on the edge of the grid indicates how many positive (+) and negative (–) poles are in the corresponding row or column.
- Your task is to label the dominos!

In [ ]:
def readGrid(path):
    with open(f"../assets/data/Magnetic/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        c_pos = list(map(int,f.readline().strip().split(" "))) 
        c_neg = list(map(int,f.readline().strip().split(" "))) 
        r_pos = list(map(int,f.readline().strip().split(" "))) 
        r_neg = list(map(int,f.readline().strip().split(" "))) 
        nums = [c_pos, c_neg, r_pos, r_neg]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        for idx, g in enumerate(res):
            res[idx] = list(map(str, g))
        return int(m), int(n), nums, res

if __name__ == "__main__":
    m, n, nums, grid = readGrid("5_12x12")
    # print(grid)
    # print(nums)
    for num in nums:
        print(num)

    

[4, 4, 5, 4, 4, 5, 5, 4, 5, 5, 3, 6]
[5, 5, 5, 4, 2, 5, 4, 5, 5, 6, 3, 5]
[5, 4, 4, 4, 4, 6, 6, 2, 5, 4, 6, 4]
[4, 6, 3, 5, 4, 5, 4, 5, 3, 5, 4, 6]


In [15]:
from ortools.sat.python import cp_model as cp

def MagneticSolver(m, n, nums, grid):
    model = cp.CpModel()
    x = dict()
    mag_loc_dict = dict()
    get_opposite = dict()
    for i in range(m):
        for j in range(n):
            if grid[i][j] not in mag_loc_dict:
                mag_loc_dict[grid[i][j]] = [[i,j]]
            else:
                mag_loc_dict[grid[i][j]].append([i,j])
            x[i, j, 1] = model.NewBoolVar(name = f"x[{i},{j},0]") 
            # 表示正极
            x[i, j, 2] = model.NewBoolVar(name = f"x[{i},{j},1]")
            # 表示负极
            x[i, j, 3] = model.NewBoolVar(name = f"x[{i},{j},2]")
            # 表示中性
    for k, v in mag_loc_dict.items():
        pos1, pos2 = v[0], v[1]
        get_opposite[pos1[0], pos1[1]] = pos2
        get_opposite[pos2[0], pos2[1]] = pos1
    
    # 约束格点数量
    c_pos = nums[0]
    c_neg = nums[1]
    r_pos = nums[2]
    r_neg = nums[3]
    
    for idx, num in enumerate(c_pos):
        model.Add(sum(x[i, idx, 1] for i in range(m)) == num)
    for idx, num in enumerate(c_neg):
        model.Add(sum(x[i, idx, 2] for i in range(m)) == num)
    for idx, num in enumerate(r_pos):
        model.Add(sum(x[idx, i, 1] for i in range(n)) == num)
    for idx, num in enumerate(r_neg):
        model.Add(sum(x[idx, i, 2] for i in range(n)) == num)
    
    # 约束每个格点只有一个状态
    for i in range(m):
        for j in range(n):
            model.Add(sum(x[i, j, k] for k in range(1, 4)) == 1)
    
    # 限制: 如果一个是中性, 另一个也是中性
    for i in range(m):
        for j in range(n):
            new_i, new_j = get_opposite[i, j][0], get_opposite[i, j][1]
            model.AddImplication(x[i, j, 3], x[new_i, new_j, 3])
            model.AddImplication(x[i, j, 1], x[new_i, new_j, 2])
            model.AddImplication(x[i, j, 2], x[new_i, new_j, 1])
            
            directions = [(1,0), (-1,0),(0,1),(0,-1)]
            for d_x, d_y in directions:
                if i + d_x >= 0 and i + d_x < m and j + d_y >= 0 and j + d_y < n:
                    if i + d_x != new_i or j + d_y != new_j:
                        # 说明不是邻居
                        # print(i, j, i + d_x, j + d_y)
                        # pass
                        model.AddBoolOr([x[i, j, 2].Not(), x[i + d_x, j + d_y, 2].Not()])

                        model.AddBoolOr([x[i, j, 1].Not(), x[i + d_x, j + d_y, 1].Not()])
                    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(m):
            for j in range(n):
                if solver.Value(x[i, j, 1]) == 1:
                    print("+", end = " ")
                elif solver.Value(x[i, j, 2]) == 1:
                    print("-", end = " ")
                else:
                    print("*", end = " ")
            print()
    else:
        print("OK!")
    

if __name__ == "__main__":
    m, n, nums, grid = readGrid("5_12x12")
    MagneticSolver(m, n, nums, grid)

+ - + - + - + - * * * + 
- * - + - + - + * - + - 
* * * * + - + * - + - + 
* - + - * + - * + - + - 
- + - + * * * * - + - + 
+ - + - + * + - + - + - 
- + - + * + - + - + * + 
+ - * * * - * - + - * - 
* * + - * + * + - + - + 
- + - + * - + - + - * * 
+ - + * - + - + - + * + 
- + - * + - + - + - * - 
